In [32]:
file_path = '../OpenSamples/Data/bnb_WithWire_16.h5'


In [212]:
import h5py
import numpy as np

# Open the HDF5 file
with h5py.File(file_path, 'r') as hf:
    # Access the "wire_table" group
    wire_table_group = hf['wire_table']

    nEvt = 1
    # Define the start and end indices for the subset you want to load

    # Load the subsets of the data using slicing and convert them to DataFrames
    event_ids = wire_table_group['event_id'][nEvt*8256]
    planeadcs =[[],[],[]]
    planeadcs[0] = wire_table_group['adc'][0+nEvt*8256:2400+nEvt*8256]
    planeadcs[1] = wire_table_group['adc'][2400+nEvt*8256:4800+nEvt*8256]
    planeadcs[2] = wire_table_group['adc'][4800+nEvt*8256:8256+nEvt*8256]

In [213]:
print("number of plane 0 wires=",len(planeadcs[0]))
print("number of plane 1 wires=",len(planeadcs[1]))
print("number of plane 2 wires=",len(planeadcs[2]))
print("number of ADC reads=",len(planeadcs[0][0]))
aspratio = len(planeadcs[2])/len(planeadcs[1])
print("plane2 vs plane0/1 aspect ratio = ",aspratio)

number of plane 0 wires= 2400
number of plane 1 wires= 2400
number of plane 2 wires= 3456
number of ADC reads= 6400
plane2 vs plane0/1 aspect ratio =  1.44


In [214]:
import numpy as np
import matplotlib.pyplot as plt

ntimeticks=6400
nplanes=3


from skimage.measure import block_reduce
f_downsample = 6
for p in range(0,nplanes):
    planeadcs[p] = block_reduce(planeadcs[p], block_size=(1,f_downsample), func=np.sum)

adccutoff = 10.*f_downsample/6.
adcsaturation = 100.*f_downsample/6.
for p in range(0,nplanes):
    planeadcs[p][planeadcs[p]<adccutoff] = 0
    planeadcs[p][planeadcs[p]>adcsaturation] = adcsaturation

zmax = adcsaturation

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20, 15), dpi=600)

im1 = ax1.imshow(planeadcs[0].T,vmin=0,vmax=zmax,origin='lower',cmap='jet')
ax1.set_title("Plane 0")
ax1.set_xlabel("Wire")
ax1.set_ylabel("Time Tick")

im2 = ax2.imshow(planeadcs[1].T,vmin=0,vmax=zmax,origin='lower',cmap='jet')
ax2.set_title("Plane 1")
ax2.set_xlabel("Wire")

im3 = ax3.imshow(planeadcs[2].T,vmin=0,vmax=zmax,origin='lower',cmap='jet')
ax3.set_title("Plane 2")
ax3.set_xlabel("Wire")

plt.tight_layout()